In [1]:
! wget http://www.gutenberg.org/files/2600/2600-0.txt

--2019-06-02 15:28:15--  http://www.gutenberg.org/files/2600/2600-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359545 (3.2M) [text/plain]
Saving to: ‘2600-0.txt’

2600-0.txt          100%[===================>]   3.20M  2.04MB/s    in 1.6s    

2019-06-02 15:28:17 (2.04 MB/s) - ‘2600-0.txt’ saved [3359545/3359545]



In [100]:
import re

# To make war and peace consistent with the text in the symbols file we have to 
# map certain unicode characters to their semantically identical ascii 
# equivalents 

# ” right quotes, U+201D
# “ left quotes U+201C 
# both map to regular quotation marks : U+0022

# ‘ left single quotation mark, U+2018
# ’ right signle quotation mark, U+2019
# both map to regular quotation mark: U+0027

with open("2600-0.txt") as train_file:
    text = train_file.read()
    text = re.sub('\n|\t','', text)
    text = text.replace("”", '"').replace("“",'"').replace("‘", "'").replace("’", "'")
    
#loading allowed symbols
with open("symbols.txt") as symbol_file:
    symbols = []
    for line in symbol_file:
        symbols.append(line[:-1]) # can't use strip as one character is white-space

## Parameter estimates

$p(s_i)$: Treat this is an estimate of a categorical distribution such that a letter $s$ (one hot encoded vector) drawn from it \begin{equation}\bar{s} \sim \textbf{Cat}(\bar{p}) \end{equation} with $\bar{p}$ a vector of category probabilities. The maximum likelihood estimate of $p$ given observed draws is $p_i = \frac{c_i}{\sum c_i} $ i.e. the count of category $i$ divided by the total count. In small data settings is is conventional to use a dirichelet prior to ensure, for example, that none of the probabilities are zero. Here, we have a very large training corpus and ignore this.

For the case of calculating the conditional distributions $p(s_n | s_{n-1})$, we use the same model, but parameterise a different distribution for each $s_{n-1}$. Due due the limitation of data here, we use a dirichelet prior on p with $\alpha_i =1 $. \begin{equation}p_{ij} = \frac{c_{ij} + 1}{\sum_j c_{ij} + N}\end{equation} with $N$ the number of unique characters and $c_{ij}$ the counts of letter $s_i$ following letter $s_j$.




## 1. Table with letter frequencies

In [266]:
from collections import Counter 
import numpy as np
import pandas as pd

counts = Counter(text)
counts = {s: counts[s] for s in symbols}
count_total = sum(counts.values())
p_i = np.array([i / count_total for s, i in counts.items() ])
symbols = [s for s in counts.keys()]

def generate_conditional_counts(text):
    ## encode a 2 by 2 array such that p(s_i | s_{i-1}) = arr[i,j]
    conditional_counts = np.ones([len(symbols), len(symbols)])
    for i in range(1, len(text)):
        if text[i] in symbols and text[i-1] in symbols:
            conditional_counts[symbols.index(text[i]), symbols.index(text[i-1])]+=1    
    
    return conditional_counts / np.sum(conditional_counts, axis=0)

p_ij = generate_conditional_counts(text)

In [105]:
pd.DataFrame.from_dict({"Letter": letter_types, "Frequency": p_i})

,Frequency,Letter
0,1.010721e-01,e
1,1.295109e-02,","
2,1.002394e-02,.
3,3.246619e-07,]
4,2.081635e-02,u
5,5.335040e-02,i
6,3.775169e-02,d
7,3.295319e-04,:
8,3.110846e-02,l
9,5.191702e-02,s


## 2. Are the latent variables independent?

No, imagine at 2 letter alphabet $[0,1]$. Then $\sigma(0) = 0 \rightarrow \sigma(1) = 1$



## 3. Joint probabilities

\begin{align}p(s_1, ... s_n , e_1 ... e_n | \sigma) &= p(e_1 ... e_n | s_1, ... s_n, \sigma) p(s_1, \ ... s_n) \\&=\prod_i\delta_{\sigma(s_i),e_i}p(s_1, \ ... s_n)\end{align}

$\delta_{ij}$ is the kroenecker delta

Alternatively, let $\alpha = \sigma^{-1}$ then 
\begin{align}p(s_1, ... s_n , e_1 ... e_n | \alpha) &= p(\alpha(e_1), \ ... \alpha(e_n))\end{align}
which is fine as $\sigma$ is bijective



## 4. Proposal and acceptance probabilities

Imagine $s_1, ... s_n , e_1 ... e_n$, are vectors that are one hot encoded to give letters. $\alpha$ is effectively an $n_\textbf{letters} \times n_\textbf{letters}$ permutation matrix that maps the encrypted symbols back to decrypted. The propoal distribution amounts to applying a random permutation matrix to $\alpha$ that swaps any 2 rows. Ths is a reversible operation, so the acceptance probability is \begin{equation} min\left(1, \frac{p(\alpha_m(e_1), \ ... \alpha_m(e_n)) }{ p(\alpha_{m-1}(e_1), \ ... \alpha_{m-1}(e_n))}\right) \end{equation}


## 5. Implementing the mh algorithm

This is not a very general MH algorithm, and could use some re-factoring. It is implemented mostly using a numpy vectorized approach, although there is an indicated list comprehension that is likely to be wasteful. Future would could include making this more general and writing unit tests. 



In [271]:
from operator import mul
import random as rd

with open("message.txt") as message_file:
    message_text = message_file.read().strip()

def calculate_logp(message, p_i, p_ij):
    """ Calculate the log probability of the message in one-hot format
    """
    
    letters = np.argmax(message,axis=1)
    p_0 = np.log(p_i[letters[0]])
    shifted_letters = np.roll(letters,1)[1:].reshape(len(message) - 1,1)
    letters = letters[1:].reshape(len(message) - 1,1)
    coords = np.concatenate([letters,shifted_letters], axis=1)
    # Should really vectorize this too. I will if the algorithm is too slow.
    res = np.sum(np.asarray([np.log(p_ij[li,li_m1]) for li, li_m1 in coords]))
    
    return res + p_0  

def apply_row_permuation(m, ix_1, ix_2):
    c = np.array(m[ix_1,:])
    m[ix_1,:] = m[ix_2,:]
    m[ix_2,:] = c
    
def one_hot_message(message):
    message_conv = np.zeros((len(message), len(symbols)))
    # one hot encode in the correct way
    for ix, i in enumerate(message):
        message_conv[ix, symbols.index(i)] = 1
    assert np.sum(message_conv) == len(message)
    return message_conv

def encoded_to_str(message_conv):
    return "".join([symbols[i] for i in np.argmax(message_conv,axis=1)])

# in lieu of more rigorous unit tests..
assert "fdfvfdnjkefndvnkfneff849y589y3hernvjlidvdds*" == encoded_to_str(one_hot_message("fdfvfdnjkefndvnkfneff849y589y3hernvjlidvdds*"))

def decode(message_conv, p_i, p_ij, n_iters=20000):
    
    a = np.eye(len(symbols)) # start with the identity mapping
    rows_to_swap = rd.randint(0,len(symbols)-1), rd.randint(0,len(symbols)-1)
 

    cur_log_p = -100000
    for i in range(n_iters):
        
        decoded = message_conv.dot(a)
        proposal_log_p = calculate_logp(decoded,p_i, p_ij)
        
        # update step
        if proposal_log_p > cur_log_p or rd.uniform(0,1) < np.exp(proposal_log_p - cur_log_p):
            cur_log_p = proposal_log_p
        else: # put the permutation matrix back to how it was before
            apply_row_permuation(a, rows_to_swap[0], rows_to_swap[1])
        
        # generate a new sample from the permutation distributions
        rows_to_swap = rd.randint(0,len(symbols)-1), rd.randint(0,len(symbols)-1)
        apply_row_permuation(a, rows_to_swap[0], rows_to_swap[1])

        print (i,cur_log_p,end="\r")
        
    return a
        
        
inv_transform = decode(one_hot_message(message_text), p_i, p_ij)
decoded_message = encoded_to_str(one_hot_message(message_text).dot(inv_transform))

print (decoded_message)

in my younger and more vulnerable years my fat*er gave me some advice t*at i've been turning over in my mind ever since. "w*enever you feel like criticizing any one," *e told me, "just remember t*at all t*e people in t*is world *aven't *ad t*e advantages t*at you've *ad." *e didn't say any more but we've always been unusually communicative in a reserved way, and i understood t*at *e meant a great deal more t*an t*at. in consequence i'm inclined to reserve all judgments, a *abit t*at *as opened up many curious natures to me and also made me t*e victim of not a few veteran bores. t*e abnormal mind is quick to detect and attac* itself to t*is quality w*en it appears in a normal person, and so it came about t*at in college i was unjustly accused of being a politician, because i was privy to t*e secret griefs of wild, unknown men. most of t*e confidences were unsoug*t--frequently i *ave feigned sleep, preoccupation, or a *ostile levity w*en i realized by some unmistakable sign t*at an intim

## Saving Text for posterity:

in my younger and more vulnerable years my fat*er gave me some advice t*at i've been turning over in my mind ever since. "w*enever you feel like criticizing any one," *e told me, "just remember t*at all t*e people in t*is world *aven't *ad t*e advantages t*at you've *ad." *e didn't say any more but we've always been unusually communicative in a reserved way, and i understood t*at *e meant a great deal more t*an t*at. in consequence i'm inclined to reserve all judgments, a *abit t*at *as opened up many curious natures to me and also made me t*e victim of not a few veteran bores. t*e abnormal mind is quick to detect and attac* itself to t*is quality w*en it appears in a normal person, and so it came about t*at in college i was unjustly accused of being a politician, because i was privy to t*e secret griefs of wild, unknown men. most of t*e confidences were unsoug*t--frequently i *ave feigned sleep, preoccupation, or a *ostile levity w*en i realized by some unmistakable sign t*at an intimate revelation was quivering on t*e *orizon--for t*e intimate revelations of young men or at least t*e terms in w*ic* t*ey express t*em are usually plagiaristic and marred by obvious suppressions. reserving judgments is a matter of infinite *ope. i am still a little afraid of missing somet*ing if i forget t*at, as my fat*er snobbis*ly suggested, and i snobbis*ly repeat a sense of t*e fundamental decencies is parcelled out unequally at birt*.

# 6. Ergodicity?

A finite irreducible markov chain, meaning that there are nonzero transition probabilities between each state, generated by the MH algorithm is ergodic. Without the dirichelet prior, one could conceive of a local mode that yielded values of $p(s_n | s_{n-1}) = 0$ as this has never been observed in training data.

Another issue is that by only swapping pairwise rows, there will be a large number of states that will have zero probability transitions. One could solve this by just trying out an entire new permutation at each step, though this is likely to be very slow to find optimal regions. A better approach might be to select from a distribution the number of rows of the permutation matrix to permute at each timestep, with larger values having low probability.